Let's play around with the graphviz backend.

### Shift register

This example is taken from the shift_register notebook.

In [1]:
from magma import *
from magma.primitives import DefineRegister

N = 4
Register4 = DefineRegister(4)
T = Bits(N)

class ShiftRegister(Circuit):
    name = "ShiftRegister"
    IO = ["I", In(T), "O", Out(T), "CLK", In(Bit)]
    @classmethod
    def definition(io):
        regs = [Register4() for _ in range(N)]
        [wire(io.CLK, reg.clk) for reg in regs]  # TODO: Clean up this clock wiring
        wire(io.I, regs[0].D)
        braid(regs, foldargs={"D":"Q"})
        wire(regs[-1].Q, io.O)


AttributeError: 'coreir_reg_P' object has no attribute 'D'

Now let's visualize our `ShiftRegister` type.

In [ ]:
ShiftRegister

For some separate reason, the visualization is slightly broken (outputs aren't wired up) for a placed circuit / instance (?) like `ShiftRegister()` below.

It's not graphviz-specific, I think: the same issue shows up in firrtl and verilog compile output.

In [ ]:
ShiftRegister()

### Simple ALU

Another example, taken from the Chisel examples folder.

In [ ]:
from functools import reduce
from magma.bitutils import *

def one_hot_mux(conds, inputs):
    outputs = []
    for cond, inp in zip(conds, inputs):
        outputs.append(inp & [cond for _ in range(len(inp))])
    return list(reduce(lambda x, y: x | y, outputs))


def uint(*ts, num_bits=None):
    bits = int2seq(ts[0], num_bits)
    return UInt(num_bits, BitOut)(*bits)


class SimpleALU(Circuit):
    name = "SimpleALU"
    IO = ["a", In(UInt(4)), "b", In(UInt(4)), "opcode", In(UInt(2)), "out", Out(UInt(4))]
    
    @classmethod
    def definition(io):
        is_op0 = io.opcode == uint(0, num_bits=2)
        is_op1 = io.opcode == uint(1, num_bits=2)
        is_op2 = io.opcode == uint(2, num_bits=2)
        is_op3 = io.opcode == uint(3, num_bits=2)
        op0_out = io.a + io.b
        op1_out = io.a - io.b
        op2_out = io.a
        op3_out = io.b
        wire(io.out, one_hot_mux([is_op0, is_op1, is_op2, is_op3], [op0_out, op1_out, op2_out, op3_out]))

In [ ]:
SimpleALU